In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import urllib
params ='DRIVER={SQL Server};SERVER=CL-AMILAH\AMILA;DATABASE=GSH;UID=Amila;Trusted_Connection=yes;'             
params = urllib.parse.quote_plus(params)

from sqlalchemy import create_engine
db = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

# Load sales data

In [3]:
sql = '''
SELECT inv.[ID] ,inv.SiteUID, [Date] , [OutletUID] ,ot.Name AS outlet ,ot.ProvinceUID,pr.Description AS province ,ot.DistrictUID ,  di.Description AS district 
, ot.AreaUID , ar.Description AS Area , inv.[AgentUID] ,Concat(emp.FirstName ,' ' , emp.MiddleInitials , ' ', emp.LastName ) AS AgentName , inv.[DistributorUID] , d.Name AS distributor, [ItemUID] ,it.Description AS item, [UnitQty] 
, [UnitPrice] , [RetailUnitPrice] , [FreeQty] , [FreeCompanyQty] , [GrossValue]
,datename(YEAR, Date) AS Year
,datename(month, Date) AS MonthName
,MONTH(Date) AS Month
,datename(DAY, Date) AS Day
,DATEPART(DW,Date) AS WeekDay
,datename(WEEKDAY, Date) AS DateName
,DATEPART (hh,Date) as Hour

  FROM [GSH].[dbo].[vw_Invoice] inv
  LEFT JOIN [GSH].[dbo].[vw_InvoiceLine] invl
  ON inv.siteUID = invl.TxnInvoice_SiteUID AND inv.uid = invl.TxnInvoiceUID	
  LEFT JOIN [GSH].[dbo].[Item] it
  ON invl.ItemUID = it.UID
  LEFT JOIN [GSH].[dbo].[Outlet] ot
  ON inv.OutletUID = ot.UID
  LEFT JOIN [GSH].[dbo].[Area] pr
  ON ot.ProvinceUID = pr.UID
  LEFT JOIN [GSH].[dbo].[Area] di
  ON ot.DistrictUID = di.UID
  LEFT JOIN [GSH].[dbo].[Area] ar
  ON ot.AreaUID = ar.UID
  LEFT JOIN [GSH].[dbo].[Distributor] d
  ON inv.DistributorUID = d.UID
  LEFT JOIN [GSH].[dbo].[avLogin] avlg
  ON inv.AgentUID = avlg.BusinessUserUID
  LEFT JOIN [GSH].[dbo].[Employee] emp
  ON avlg.EmployeeUID = emp.UID
  WHERE inv.mpt_TypeEnum IN ( 21 , 44, 14, 1) AND inv.ExpiryDate IS NULL AND inv.SubmittedDate IS NOT NULL
  AND Date >= '2018-11-01' AND Date <= '2020-03-01' ;
'''

invoice_All = pd.read_sql_query(sql, db)

In [4]:
#disply all the columns
pd.set_option('display.max_columns', None)

In [5]:
invoice_All.head()

,ID,SiteUID,Date,OutletUID,outlet,ProvinceUID,province,DistrictUID,district,AreaUID,Area,AgentUID,AgentName,DistributorUID,distributor,ItemUID,item,UnitQty,UnitPrice,RetailUnitPrice,FreeQty,FreeCompanyQty,GrossValue,Year,MonthName,Month,Day,WeekDay,DateName,Hour
0,IN0066000105,0,2018-11-02 14:55:18.963,2533,Union Pharmacy,16.0,Southern Province,18.0,Matara,51.0,Dickwella,104,Praneeth Waruna,14,Unichemist Distributor,5,Benzoyl Peroxide,3.0,368.5,420.0,0.0,0.0,1105.5,2018,November,11,2,6,Friday,14
1,CSO011600702,114,2018-11-02 15:10:01.177,3067,New Centrel Pharmacy,20.0,Western Province,23.0,Gampaha,474.0,Kadawatha,116,,22,Lucky Impex,23,Julab Tab 5mg 50's,2.0,3191.2,3638.0,0.0,0.0,6382.4,2018,November,11,2,6,Friday,15
2,CSO011600702,114,2018-11-02 15:10:01.177,3067,New Centrel Pharmacy,20.0,Western Province,23.0,Gampaha,474.0,Kadawatha,116,,22,Lucky Impex,207,Growee Syrup 120ml,1.0,653.5,745.0,0.0,0.0,653.5,2018,November,11,2,6,Friday,15
3,CSO007901569,53,2018-11-02 15:57:23.843,289,SILVER RAY PHARMACY,20.0,Western Province,23.0,Gampaha,41.0,Negombo,79,Shashikala perera,2,The West Central Pharma (Pvt) Ltd.,206,Solmux Forte Susp. 60ml,4.0,477.3,544.0,0.0,0.0,1909.2,2018,November,11,2,6,Friday,15
4,CSO008001478,56,2018-11-02 15:57:35.797,246,HIRU PHARMACY,20.0,Western Province,23.0,Gampaha,93.0,DIVULAPITIYA,80,Niralini Waruna,2,The West Central Pharma (Pvt) Ltd.,240,nitroimidazoles GEL 30mg,5.0,233.4,266.0,0.0,0.0,1167.0,2018,November,11,2,6,Friday,15


### What products are most often sold together?

In [6]:
Sold_Together =invoice_All.groupby(["ID" , "item"]).agg({'GrossValue':'sum'}).reset_index()

In [7]:
Sold_Together = Sold_Together[Sold_Together['ID'].duplicated(keep=False)]

Sold_Together['Grouped'] = Sold_Together.groupby('ID')['item'].transform(lambda x: ','.join(x))
Sold_Together_final = Sold_Together[['ID', 'Grouped']].drop_duplicates()

In [8]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in Sold_Together_final['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

for key,value in count.most_common(10):
    print(key, value)

('Cilnidipine tab 10mg ', 'Cilnidipine tab 5mg') 8892
('nitroimidazoles GEL 20mg', 'nitroimidazoles GEL 30mg') 8091
('Clobetasol G cream 15mg', 'Gentamicin\xa0Cream 15 mg') 7998
('Nifedipine 30mg', 'nitroimidazoles GEL 30mg') 7884
('Clobetasol G cream 15mg', 'benzoyl peroxide wash 60mg') 7279
('Cilnidipine tab 5mg', 'nitroimidazoles GEL 30mg') 7197
('Cilnidipine tab 5mg', 'Nifedipine 30mg') 6072
('hypoglycaemic sulfonylurea antidiabetic 80mg', 'nitroimidazoles GEL 30mg') 5733
('Clobetasol G cream 15mg', 'Gentamicin 15 mg') 5723
('Benzoyl Peroxide', 'Clobetasol G cream 15mg') 5450


In [9]:
print(Sold_Together_final['ID'].value_counts())

CSO009903000    1
CSO010003700    1
IN0068001895    1
CSO010500756    1
CSO008004328    1
               ..
CSO008803186    1
CSO007906654    1
CSO016802211    1
CSO008401648    1
CSO014801280    1
Name: ID, Length: 135529, dtype: int64
